In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
DATA_PATH = '/dlabdata1/turkish_wiki/processed_data/'

In [3]:
pageset_edits = pd.read_csv(f'{DATA_PATH}/pageset_edits_pct.csv')

In [6]:
pageset_edits['migrated'] = pageset_edits['migrated'].astype('float') 

In [11]:
pageset_edits = pageset_edits[pageset_edits['number_of_neighbors'] > 0]

In [14]:
pageset_edits.head()

,event_user_id,page_id,number_of_neighbors,centrality,migrated,migration_percentage,edits_per_day
0,25,"{815064, 222351}",5,0.000458,0.0,1.000000,0.002018
1,39,"{9312, 1067105, 1571688, 644809, 414218, 18022...",178,0.006456,0.0,0.443820,0.014127
2,47,"{1664134, 2124426, 2124428, 2124437, 61624}",12,0.000638,0.0,0.583333,0.007064
3,137,"{647363, 473637, 1631751, 16958, 8148, 1908, 9...",98,0.004934,0.0,0.520408,0.010091
4,146,"{518121, 554324, 919417}",11,0.000868,1.0,0.727273,0.003027


In [44]:
import itertools

In [55]:
columns = ['number_of_neighbors', 'centrality', 'migration_percentage', 'edits_per_day']
for L in range(0, len(columns)+1):
    for subset in itertools.combinations(columns, L):
        if len(list(subset)) > 0:
            right_hand_side = " + ".join(list(subset))
            form = f'migrated ~ {right_hand_side}'
            mod = smf.logit(formula= form,  data=pageset_edits)
            res = mod.fit()
            print(res.summary())
        

Optimization terminated successfully.
         Current function value: 0.225335
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               migrated   No. Observations:                37012
Model:                          Logit   Df Residuals:                    37010
Method:                           MLE   Df Model:                            1
Date:                Mon, 19 Apr 2021   Pseudo R-squ.:                 0.03910
Time:                        17:59:28   Log-Likelihood:                -8340.1
converged:                       True   LL-Null:                       -8679.5
Covariance Type:            nonrobust   LLR p-value:                1.254e-149
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -2.8432      0.023   -123.319      0.000      -2.888      -2.798
nu

In [39]:
mod = smf.logit(formula=f'migrated ~ migration_percentage + edits_per_day + number_of_neighbors + centrality',  data=pageset_edits)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.216870
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:               migrated   No. Observations:                37012
Model:                          Logit   Df Residuals:                    37007
Method:                           MLE   Df Model:                            4
Date:                Mon, 19 Apr 2021   Pseudo R-squ.:                 0.07520
Time:                        15:35:18   Log-Likelihood:                -8026.8
converged:                       True   LL-Null:                       -8679.5
Covariance Type:            nonrobust   LLR p-value:                2.256e-281
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -4.4790      0.099    -45.385      0.000      -4.672      -4.286

In [41]:
mod = smf.logit(formula='migrated ~ migration_percentage + edits_per_day + number_of_neighbors',  data=pageset_edits)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.224985
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:               migrated   No. Observations:                37012
Model:                          Logit   Df Residuals:                    37008
Method:                           MLE   Df Model:                            3
Date:                Mon, 19 Apr 2021   Pseudo R-squ.:                 0.04060
Time:                        15:36:04   Log-Likelihood:                -8327.1
converged:                       True   LL-Null:                       -8679.5
Covariance Type:            nonrobust   LLR p-value:                2.006e-152
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -2.9709      0.062    -48.011      0.000      -3.092      -2.850

In [30]:
scaled_pageset_edits = pageset_edits.copy()

In [36]:
np.exp(0.1*1.7958)

1.1967146374090887

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
names = ['number_of_neighbors','centrality', 'migrated', 'migration_percentage', 'edits_per_day']
scaled_pageset_edits[names] = scaler.fit_transform(pageset_edits[names])

In [32]:
scaled_pageset_edits

,event_user_id,page_id,number_of_neighbors,centrality,migrated,migration_percentage,edits_per_day
0,25,"{815064, 222351}",0.000143,0.004101,0.0,1.000000,0.000000
1,39,"{9312, 1067105, 1571688, 644809, 414218, 18022...",0.006307,0.057835,0.0,0.443820,0.000054
2,47,"{1664134, 2124426, 2124428, 2124437, 61624}",0.000392,0.005718,0.0,0.583333,0.000022
3,137,"{647363, 473637, 1631751, 16958, 8148, 1908, 9...",0.003456,0.044201,0.0,0.520408,0.000036
4,146,"{518121, 554324, 919417}",0.000356,0.007778,1.0,0.727273,0.000004
...,...,...,...,...,...,...,...
39460,964073,{700291},0.000071,0.002294,0.0,1.000000,0.000000
39461,964120,{283827},0.000143,0.003125,0.0,0.400000,0.000004
39462,964121,{1149349},0.002850,0.032741,0.0,0.518519,0.000000
39464,964213,{1679269},0.000214,0.004350,0.0,0.714286,0.000013


In [43]:
mod = smf.logit(formula='migrated ~ migration_percentage + edits_per_day',  data=scaled_pageset_edits)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.226978
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:               migrated   No. Observations:                37012
Model:                          Logit   Df Residuals:                    37009
Method:                           MLE   Df Model:                            2
Date:                Mon, 19 Apr 2021   Pseudo R-squ.:                 0.03210
Time:                        17:47:42   Log-Likelihood:                -8400.9
converged:                       True   LL-Null:                       -8679.5
Covariance Type:            nonrobust   LLR p-value:                1.040e-121
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -2.7234      0.057    -48.126      0.000      -2.834      -2.61